[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/costaalv/descobrindo_caos/blob/main/notebooks/08_lente_descoberta.ipynb)

# 8 · A Ótica Adequada: Por Que a Escala Logarítmica É Estrutural

**Registro observacional associado ao livro**  
*Descobrindo o Caos nos Números — Como a ordem emerge quando mudamos a forma de observar*  
© Alvaro Costa, 2025  

Este notebook faz parte de uma sequência canônica de registros computacionais.  
Ele não introduz hipóteses, conjecturas ou modelos interpretativos novos.

Seu objetivo é exclusivamente **registrar** o comportamento de estruturas aritméticas sob um regime  
de observação explícito, determinístico e reproduzível.

A leitura conceitual completa encontra-se no livro. Este notebook documenta apenas o experimento  
correspondente.

**Licença:** Creative Commons BY–NC–ND 4.0  
É permitida a leitura, execução e citação.  
Não é permitida a modificação, redistribuição adaptada ou uso comercial independente.

---

## 1. O Mistério das Duas Músicas

Nos capítulos anteriores, celebramos a emergência da "música" da GOE a partir do nosso operador $ M $. No entanto, uma análise mais  
atenta revela um mistério: dependendo de como olhamos para a reta numérica, a mesma partitura soa diferente.

Quando usamos uma amostragem **linear**, o espectro de $ M $ se aproxima de uma estatística **Poisson**. Já sob uma amostragem  
**logarítmica**, surge claramente a **GOE**. Por que a mesma metodologia, aplicada à mesma região, produz duas "músicas" tão distintas?  
Seria um artefato?

A resposta é não. Estamos diante de um fenômeno fundamental: a forma como olhamos o sistema altera o que o sistema revela. A chave está  
na escala natural dos próprios números primos.

---

## 2. As Duas Lentes e a Escala Natural dos Primos

A escala que naturalmente se ajusta aos primos é a **logarítmica**, como Gauss observou ao notar que a densidade de primos em torno de um  
número $ x $ é aproximadamente $ 1/\log(x) $. Para enxergar a estrutura global dos primos, precisamos de uma régua que se expande com eles  
— e essa régua é a escala logarítmica.

Os métodos de amostragem, portanto, funcionam como **duas lentes**: uma "calibrada" para essa escala e outra que a ignora.

---

### 🔹 A Lente "Panorâmica" (Amostragem Logarítmica)

Esta é a lente natural, sintonizada com o ritmo dos primos. Ao amostrar pontos em intervalos logarítmicos, olhamos o universo numérico com a  
régua certa — uma régua que cresce junto com o próprio espaço.

- **O que a lente vê:** Nessa visão panorâmica, a função $ \Delta_\pi(x) $ revela suas flutuações harmônicas e ruídos estruturais.  
- **A consequência:** A matriz $ M $ resultante é de alta complexidade, semelhante a uma matriz aleatória real simétrica, e o seu espectro exibe  
a repulsão de níveis característica da **GOE (Gaussian Orthogonal Ensemble)**.

---

### 🔹 A Lente "Microscópio" (Amostragem Linear)

Esta lente aplica uma régua rígida, sem se ajustar à escala logarítmica. É como observar um relevo de montanhas com uma lupa: tudo parece plano,  
porque a lente vê apenas um fragmento do mundo.

- **O que a lente vê:** Dentro de uma região tão estreita, a tendência global desaparece e \( \Delta_\pi(x) \) parece estável e suave. As flutuações  
locais tornam-se quase independentes — um regime de **descorrelação**.  
- **A consequência:** A matriz $ M $ construída com essa lente é de baixa variabilidade, e o seu espectro se comporta como um sistema de eventos  
independentes — uma estatística de **Poisson**.

---

### 🔹 Tabela Comparativa

| Característica | Amostragem Linear (Microscópio) | Amostragem Logarítmica (Panorâmica) |
| :--- | :--- | :--- |
| **Alinhamento** | Ignora a escala natural dos primos | Alinhada com a escala natural (Gauss) |
| **Visão da Função** | Região localmente "plana" | Flutuações globais e ruído harmônico |
| **Complexidade da Matriz** | Baixa, ordenada, previsível | Alta, complexa, pseudo-aleatória |
| **Resultado Espectral** | **Poisson** | **GOE (Gaussian Orthogonal Ensemble)** |

---

## 3. Laboratório da Escala: A Conexão com o Código

A célula de código abaixo demonstra o efeito de forma direta.  
Um detalhe técnico é fundamental para revelar a estrutura de Poisson: a função `local_normalize_spacings`.

Como a lente linear enxerga apenas uma região "quase plana", a densidade dos autovalores pode variar ligeiramente.  
A normalização local age como um **ajuste de foco fino** — recalibra cada espaçamento de acordo com o seu entorno imediato, restaurando a visão  
adequada do regime Poisson.

Esse procedimento não cria o padrão; ele o revela.  
Ao aplicar `local_normalize_spacings`, observamos que a distribuição Poisson aparece com nitidez — provando que ela é uma **característica real  
do sistema nesta escala**, e não um artefato numérico.


In [1]:
# Requisitos: pandas, matplotlib, numpy, ipywidgets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import time

# --- Funções de Geração de Dados e Matriz ---
def generate_pi_data(n: int) -> np.ndarray:
    """Gera um array com todos os primos até n usando um crivo otimizado."""
    if n < 2: return np.array([], dtype=np.int64)
    size = (n - 1) // 2; sieve = np.ones(size, dtype=bool)
    limit = int(np.sqrt(n)) // 2
    for i in range(limit):
        if sieve[i]:
            p = 2 * i + 3; start = (p*p - 3) // 2
            sieve[start::p] = False
    indices = np.where(sieve)[0]; odd_primes = 2 * indices + 3
    return np.concatenate((np.array([2], dtype=np.int64), odd_primes))

def get_delta_pi_for_points(x_points, primes):
    """Calcula Δπ(x) para um array de pontos x usando uma lista de primos pré-calculada."""
    x_int = np.floor(x_points).astype(int)
    pi_x = np.searchsorted(primes, x_int, side='right')
    pi_x_div_2 = np.searchsorted(primes, x_int // 2, side='right')
    return pi_x - 2 * pi_x_div_2
    
def generate_cos_matrix(fx_values, x_values):
    """Gera a matriz M a partir dos vetores F(x) e x."""
    fx = fx_values.astype(np.float64); x = x_values.astype(np.float64)
    x[x <= 0] = 1e-12; logx = np.log(x)
    C = np.cos(np.outer(fx, logx)); M = C + C.T
    std_dev = M.std()
    if std_dev > 0:
        M -= M.mean()
        M /= std_dev
    return 0.5 * (M + M.T)

# bulk fixo em 90% central (alpha = 0.10)
# janela local fixa para unfolding (não otimizada) w = 21
def local_normalize_spacings(lam, alpha=0.10, w=21):
    """
    Normaliza os espaçamentos pela sua média local (unfolding).
    Esta é a chave para visualizar corretamente a estatística de Poisson.
    """
    N = lam.size
    # Pega o "bulk" (meio) do espectro para evitar efeitos de borda
    k0, k1 = int(alpha * N), int((1 - alpha) * N)
    lam_bulk = np.sort(lam)[k0:k1]
    
    s = np.diff(lam_bulk)
    s = s[s > 0]
    
    if len(s) < w: return s / s.mean() if s.mean() > 0 else s

    # Usa uma média móvel para encontrar a densidade local de estados
    w = int(w)
    if w % 2 == 0: w += 1 # A janela deve ser ímpar
    pad = w // 2
    s_padded = np.pad(s, (pad, pad), mode='reflect')
    local_mean = np.convolve(s_padded, np.ones(w)/w, mode='valid')
    
    # Evita divisão por zero
    local_mean[local_mean == 0] = 1.0
    
    return s / local_mean

# --- A Função Interativa Principal ---
def scale_comparison_lab(N=2048, log_X0=8, span=2.4):
    
    X0 = int(10**log_X0)
    
    # --- Preparação dos Dados ---
    max_x_log = int(np.ceil(X0 * np.exp(span/2)))
    max_x_linear = X0 + N
    max_x_needed = max(max_x_log, max_x_linear)
    pi_x_full = generate_pi_data(max_x_needed)

    fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharey=True) 
    
    # --- Gráfico da Esquerda: Amostragem Linear (Poisson) ---
    print("\n--- Processando Escala Linear ---")
    x_linear = np.arange(X0, X0 + N)
    fx_linear = get_delta_pi_for_points(x_linear, pi_x_full)
    
    M_linear = generate_cos_matrix(fx_linear, x_linear)
    lam_linear, _ = np.linalg.eigh(M_linear)
    # USA A NORMALIZAÇÃO LOCAL PARA OBTER POISSON
    s_unfolded_linear = local_normalize_spacings(lam_linear)

    # --- Gráfico da Direita: Amostragem Logarítmica (GOE) ---
    print("\n--- Processando Escala Logarítmica ---")
    x_log = np.exp(np.linspace(np.log(X0) - span/2, np.log(X0) + span/2, N))
    fx_log = get_delta_pi_for_points(x_log, pi_x_full)

    M_log = generate_cos_matrix(fx_log, x_log)
    lam_log, _ = np.linalg.eigh(M_log)
    # Para GOE, a normalização pela média global já funciona bem
    s_log = np.diff(np.sort(lam_log)); s_log = s_log[s_log > 0]
    s_unfolded_log = s_log / s_log.mean()

    # --- Plots Comparativos ---
    s_grid = np.linspace(0, 4, 200)
    pdf_goe = (np.pi * s_grid / 2) * np.exp(-np.pi * s_grid**2 / 4)
    pdf_poisson = np.exp(-s_grid)
    
    # Plot da Esquerda
    ax = axes[0]
    ax.hist(s_unfolded_linear, bins='auto', density=True, alpha=0.75, label='Dados (Linear)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='Teoria GOE')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Teoria Poisson')
    ax.set_title(f'a) Escala Linear → Regime Não Correlacionado', fontsize=14)
    ax.set_xlabel('s (Espaçamento Normalizado Localmente)'); ax.set_ylabel('Densidade')
    ax.set_xlim(0, 4); ax.legend(loc='upper right')
    
    # Plot da Direita
    ax = axes[1]
    ax.hist(s_unfolded_log, bins='auto', density=True, alpha=0.75, label='Dados (Log)')
    ax.plot(s_grid, pdf_goe, 'r--', lw=2, label='Teoria GOE')
    ax.plot(s_grid, pdf_poisson, 'g:', lw=3, label='Teoria Poisson')
    ax.set_title(f'b) Escala Logarítmica → Regime Correlacionado', fontsize=14)
    ax.set_xlabel('s (Espaçamento Normalizado Globalmente)'); ax.legend(loc='upper right')
    ax.set_xlim(0, 4)
    
    fig.suptitle(f"Comparação Visual do Efeito da Escala em X₀ = {X0:g}", fontsize=18, weight='bold')
    fig.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# --- Widget Interativo ---
interact(scale_comparison_lab, 
         N=widgets.Dropdown(options=[512, 1024, 2048], value=2048, description='N:'),
         log_X0=widgets.IntSlider(min=5, max=8, step=1, value=8, description='X₀=10^', continuous_update=False),
         span=widgets.FloatSlider(min=1.0, max=4.0, step=0.1, value=2.4, description='Span (Log):')
        );


interactive(children=(Dropdown(description='N:', index=2, options=(512, 1024, 2048), value=2048), IntSlider(va…

Com a prova experimental em mãos de que as duas lentes produzem músicas diferentes, estamos prontos para aprofundar. No próximo capítulo, vamos analisar a "ótica" da nossa lente logarítmica, mergulhando na matemática que explica por que ela não apenas funciona, mas é a única lente que poderia funcionar.